<a href="https://colab.research.google.com/github/1Proskuryakov1/VKR/blob/main/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp /content/drive/MyDrive/kaggle.json /content/

# Создание папки для хранения Kaggle API ключа и перемещение kaggle.json
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Загрузка датасета с Kaggle
!kaggle datasets download -d rmisra/news-category-dataset

# Распаковка загруженного датасета
!unzip news-category-dataset.zip

from google.colab import drive
drive.mount('/content/drive')
# Импортируем библиотеки


# Импортируем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud
from wordcloud import STOPWORDS
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


path = '/content/News_Category_Dataset_v3.json'
df = pd.read_json(path, lines=True)
df.head()

df.info()
df.describe()
df.isna().sum()
for column in df.columns:
    if df[column].dtype == 'object':
        num_empty_descriptions = (df[column].apply(len) == 0).sum()
        print(f"Количество строк, где длина '{column}' равна нулю: {num_empty_descriptions}")
        empty_headlines = df[df['headline'].apply(len) == 0]
        print(f"Количество различных категорий: {df.category.nunique()}\n")
print(f"Категории: {df['category'].unique()}\n")
print("Категория | Кол-во статей")
print(df['category'].value_counts())
df.category = df.category.replace({"HEALTHY LIVING": "WELLNESS",
              "QUEER VOICES": "GROUPS VOICES",
              "BUSINESS": "BUSINESS & FINANCES",
              "PARENTS": "PARENTING",
              "BLACK VOICES": "GROUPS VOICES",
              "THE WORLDPOST": "WORLD NEWS",
              "STYLE": "STYLE & BEAUTY",
              "GREEN": "ENVIRONMENT",
              "TASTE": "FOOD & DRINK",
              "WORLDPOST": "WORLD NEWS",
              "SCIENCE": "SCIENCE & TECH",
              "TECH": "SCIENCE & TECH",
              "MONEY": "BUSINESS & FINANCES",
              "ARTS": "ARTS & CULTURE",
              "COLLEGE": "EDUCATION",
              "LATINO VOICES": "GROUPS VOICES",
              "CULTURE & ARTS": "ARTS & CULTURE",
              "FIFTY": "OTHER",
              "GOOD NEWS": "OTHER"}
            )

# График распределения категорий
plt.figure(figsize=(12,8))
sns.countplot(y=df['category'], order=df['category'].value_counts().index)
plt.title('Распределение категорий новостей')
plt.xlabel('Количество статей')
plt.ylabel('Категория')
plt.show()

# Вычисление длины заголовков
headline_lengths = df['headline'].apply(len)

# Вычисление квантилей
quantiles = headline_lengths.quantile([0.25, 0.5, 0.75])

# Построение гистограммы
plt.figure(figsize=(12,6))
sns.histplot(headline_lengths, bins=50, kde=True)

# Добавление квантилей на график
for quantile in quantiles:
    plt.axvline(quantile, color='r', linestyle='--')
    # plt.text(quantile, plt.gca().get_ylim()[1]*0.9, f'{quantile:.2f}', color='r', ha='center')

# Настройки графика
plt.title('Распределение длины заголовков')
plt.xlabel('Длина заголовка (кол-во символов)')
plt.ylabel('Количество')
plt.show()

# Вычисление длины заголовков
description_lengths = df['short_description'].apply(len)

# Вычисление квантилей
quantiles = description_lengths.quantile([0.25, 0.5, 0.75])

# Построение гистограммы
plt.figure(figsize=(12,6))
sns.histplot(description_lengths, bins=50, kde=True)

# Добавление квантилей на график
for quantile in quantiles:
    plt.axvline(quantile, color='r', linestyle='--')
    # plt.text(quantile, plt.gca().get_ylim()[1]*0.9, f'{quantile:.2f}', color='r', ha='center')

# Настройки графика
plt.title('Распределение длины описаний')
plt.xlabel('Длина описания (кол-во символов)')
plt.ylabel('Количество')
plt.show()


from matplotlib.cm import get_cmap

# Преобразование даты в формат datetime и создание столбца с годом и месяцем
df['date'] = pd.to_datetime(df['date'])
df['year_month'] = df['date'].dt.to_period('M')

categories = df['category'].unique()
cmap = get_cmap('tab20', len(categories))

plt.figure(figsize=(16,10))
for i, category in enumerate(df['category'].unique()):
    subset = df[df['category'] == category]
    subset.groupby('year_month').size().plot(label=category, color=cmap(i))

# Получение всех уникальных значений year_month
all_year_months = df['year_month'].sort_values().unique()

# x_labels = [all_year_months[0], all_year_months[len(all_year_months) // 2], all_year_months[-1]]
num_labels = 10
x_labels = [all_year_months[i] for i in range(0, len(all_year_months), len(all_year_months) // num_labels)]


plt.title('Тренды новостей по категориям во времени', size=20)
plt.xlabel('Дата')
plt.ylabel('Количество новостей')
plt.xticks(x_labels, [label.strftime('%Y-%m') for label in x_labels], rotation=45)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()


categories = df['category'].unique()

df['year_month'] = df['date'].dt.to_period('M').astype(str)

figure, axis = plt.subplots(9, 3, figsize=(12, 15))

for i, category in enumerate(categories):
    # Получение тренда для текущей категории
    trend = df[df['category'] == category].groupby('year_month').size()

    row = i // 3
    col = i % 3

    # Построение графика для текущей категории
    axis[row, col].plot(trend)
    axis[row, col].set_title(category)

    x_labels = [trend.index[0], trend.index[len(trend) // 2], trend.index[-1]]
    axis[row, col].set_xticks(x_labels)
    axis[row, col].set_xticklabels(x_labels)

# Увеличение пространства между графиками
plt.subplots_adjust(hspace=0.75)
plt.show()

# Фильтрация данных для новостей до 2018 года
df_before_2018 = df[df['date'] < '2018-01-01']

# Создание сетки подграфиков
figure, axis = plt.subplots(9, 3, figsize=(12, 15))

for i, category in enumerate(categories):
    # Получение тренда для текущей категории и отфильтрованных данных
    trend = df_before_2018[df_before_2018['category'] == category].groupby('year_month').size()

    row = i // 3
    col = i % 3

    # Построение графика для текущей категории
    axis[row, col].plot(trend)
    axis[row, col].set_title(category)
    try:
        x_labels = [trend.index[0], trend.index[len(trend) // 2], trend.index[-1]]
        axis[row, col].set_xticks(x_labels)
        axis[row, col].set_xticklabels(x_labels)
    except:
        continue

# Увеличение пространства между графиками
plt.subplots_adjust(hspace=0.75)
plt.show()

# Фильтрация данных для новостей до 2018 года
df_before_2018 = df[df['date'] < '2018-01-01']

# Создание сетки подграфиков
figure, axis = plt.subplots(9, 3, figsize=(12, 15))

for i, category in enumerate(categories):
    # Получение тренда для текущей категории и отфильтрованных данных
    trend = df_before_2018[df_before_2018['category'] == category].groupby('year_month').size()

    row = i // 3
    col = i % 3

    # Построение графика для текущей категории
    axis[row, col].plot(trend)
    axis[row, col].set_title(category)
    try:
        x_labels = [trend.index[0], trend.index[len(trend) // 2], trend.index[-1]]
        axis[row, col].set_xticks(x_labels)
        axis[row, col].set_xticklabels(x_labels)
    except:
        continue

# Увеличение пространства между графиками
plt.subplots_adjust(hspace=0.75)
plt.show()


# Преобразование даты в формат datetime
df['date'] = pd.to_datetime(df['date'])

# Создание нового столбца с днем недели
df['day_of_week'] = df['date'].dt.day_name()

# Группировка данных по категориям и дням недели и подсчет количества публикаций
grouped_data = df.groupby(['category', 'day_of_week']).size().unstack()

# Создание графика для каждой категории
fig, axes = plt.subplots(9, 3, figsize=(12, 15))

for ax, (category, data) in zip(axes.flatten(), grouped_data.iterrows()):
    data = data.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
    data.plot(kind='bar', ax=ax, color='skyblue')
    ax.set_title(category)
    ax.set_xticklabels(data.index.str[0], rotation=0)
    ax.set_xlabel(None)

# Увеличение пространства между графиками
plt.subplots_adjust(wspace=0.25, hspace=0.75)

plt.suptitle('Частота публикаций по дням недели для различных категорий', fontsize=20)
plt.show()


# Подсчет количества статей для каждого автора
author_counts = df['authors'].value_counts()

# Определение топ-10 самых активных авторов
top_10_authors = author_counts[1:11]

# Построение столбчатой диаграммы для топ-10 самых активных авторов
plt.figure(figsize=(12, 6))
top_10_authors.plot(kind='bar', color='skyblue')
plt.title('Количество статей топ-10 самых активных авторов')
plt.xlabel('Авторы')
plt.ylabel('Количество статей')
plt.xticks(rotation=45)
plt.show()


plt.figure(figsize=(12,12))
wc = WordCloud(max_words=1000,
               min_font_size=10,
               height=600,
               width=1600,
               background_color='black',
               contour_color='black',
               colormap='plasma',
               repeat=False,
               stopwords=STOPWORDS).generate(' '.join(df.headline))

plt.title("Облако слов из всех заголовков", size=15)
plt.imshow(wc, interpolation= "bilinear")
plt.axis('off')


plt.figure(figsize=(12,12))
wc = WordCloud(max_words=1000,
               min_font_size=10,
               height=600,
               width=1600,
               background_color='black',
               contour_color='black',
               colormap='plasma',
               repeat=False,
               stopwords=STOPWORDS).generate(' '.join(df.short_description))

plt.title("Облако слов из всех описаний", size=15)
plt.imshow(wc, interpolation= "bilinear")
plt.axis('off')


fig, axes = plt.subplots(9, 3, figsize=(8, 16), subplot_kw=dict(xticks=[], yticks=[], frame_on=False))
plt.subplots_adjust(hspace=0.35)
for ax, category in zip(axes.flatten(), df['category'].unique()):
    wordcloud = WordCloud(width=500, height=300, random_state=42, max_font_size=100, background_color='black',
                         colormap='plasma', stopwords=STOPWORDS).generate(' '.join(df[df['category']==category]['headline']))
    ax.imshow(wordcloud, interpolation="bilinear")
    ax.set_title(category)

plt.show()


fig, axes = plt.subplots(9, 3, figsize=(8, 15), subplot_kw=dict(xticks=[], yticks=[], frame_on=False))
plt.subplots_adjust(hspace=0.35)
for ax, category in zip(axes.flatten(), df['category'].unique()):
    wordcloud = WordCloud(width=500, height=300, random_state=42, max_font_size=100, background_color='black',
                         colormap='plasma', stopwords=STOPWORDS).generate(' '.join(df[df['category']==category]['short_description']))
    ax.imshow(wordcloud, interpolation="bilinear")
    ax.set_title(category)

plt.show()


from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

def plot_ngrams(texts, n, num_top_ngrams, category):
    """
    Plot the top N n-grams for the given texts.

    Args:
        texts (Series): Texts to analyze.
        n (int): The n in n-grams (e.g., 1 for unigrams, 2 for bigrams).
        num_top_ngrams (int): Number of top n-grams to display.
        category (str): Category of the texts.
    """
    vectorizer = CountVectorizer(ngram_range=(n, n))
    X = vectorizer.fit_transform(texts)
    ngram_counts = X.sum(axis=0).A1

    vocabulary = vectorizer.vocabulary_
    ngrams = [gram for gram, idx in sorted(vocabulary.items(), key=lambda x: x[1])]

    sorted_indices = ngram_counts.argsort()[::-1]
    top_ngram_indices = sorted_indices[:num_top_ngrams]
    top_ngrams = [ngrams[idx] for idx in top_ngram_indices]
    top_ngram_counts = ngram_counts[top_ngram_indices]

    plt.figure(figsize=(10, 6))
    plt.barh(range(num_top_ngrams), top_ngram_counts, align='center', color='skyblue')
    plt.yticks(range(num_top_ngrams), top_ngrams)
    plt.xlabel('Count')
    plt.ylabel('N-gram')
    plt.title(f'Top {num_top_ngrams} {n}-grams - N-gram Analysis for {category} Category')
    plt.tight_layout()
    plt.show()

# Пример использования для анализа топ 10 биграмм по категориям
categories = df['category'].unique()
for category in categories:
    print(f"N-gram Analysis for {category} Category:")
    category_texts = df[df['category'] == category]['headline']
    plot_ngrams(category_texts, n=2, num_top_ngrams=10, category=category)
    df['short_description'] = df['headline'] + df['short_description']
    df = df[df['headline'] != '']
df.info()

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Убедитесь, что у вас есть необходимые данные NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

def preprocess_text(text):
    """
    Perform various preprocessing steps on the input text.

    Args:
        text (str): The input text to preprocess.

    Returns:
        str: The preprocessed text.
    """
    # Удаление специальных символов и знаков
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

   # Преобразование текста в строчные буквы
    text = text.lower()

    # Удаление  лишних пробелов
    text = re.sub(r"\s+", " ", text).strip()

    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    # Удаление неалфавитно-цифровых символов
    text = re.sub(r'\W+', ' ', text)

    # Удаление слов, начинающиеся с хэш-тегов
    text = re.sub(r'\b#\w+\b', '', text)

    # Удаление URL-адреса, начинающиеся с 'http'
    text = re.sub(r'http\S+', '', text)

    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmas = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(lemmas)

   # Набор общих слов, которые нужно удалить
    common_words = set(['news', 'article', 'report'])  # Add your common words here
    text = ' '.join([word for word in text.split() if word not in common_words])

    # Удаление цифровых символов
    text = re.sub(r'\d+', '', text)

    # Удаление слов, содержащих менее трех символовs
    text = ' '.join([word for word in text.split() if len(word) >= 3])

    return text

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    """
    Convert a Penn Treebank POS tag to a WordNet POS tag.

    Args:
        treebank_tag (str): The Penn Treebank POS tag.

    Returns:
        str: The corresponding WordNet POS tag.
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

def preprocess_text_2(text):
    """
    Apply lemmatization to the input text.

    Args:
        text (str): The input text to preprocess.

    Returns:
        str: The lemmatized text.
    """

    nltk.download('averaged_perceptron_tagger_eng')
    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return ' '.join(lemmatized_words)

df = df[:5000]

# Применение препроцессинга к столбцам 'headline' и 'short_description' в DataFrame
df['headline'] = df['headline'].apply(preprocess_text)
df['short_description'] = df['short_description'].apply(preprocess_text)

# Применение лемматизации к столбцам 'headline' и 'short_description' и создание новых столбцов
df['headline_lemmatized'] = df['headline'].apply(preprocess_text_2)
df['short_description_lemmatized'] = df['short_description'].apply(preprocess_text_2)

# Выведение на экран несколько первых строк DataFrame, чтобы убедиться, что предварительная обработка прошла как надо.
print(df[['headline', 'headline_lemmatized', 'short_description', 'short_description_lemmatized']].head())
df.sample(5)

from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

def plot_ngrams(texts, n, num_top_ngrams, category):
    """
    Plot the top N n-grams for the given texts.

    Args:
        texts (Series): Texts to analyze.
        n (int): The n in n-grams (e.g., 1 for unigrams, 2 for bigrams).
        num_top_ngrams (int): Number of top n-grams to display.
        category (str): Category of the texts.
    """
    vectorizer = CountVectorizer(ngram_range=(n, n))
    X = vectorizer.fit_transform(texts)
    ngram_counts = X.sum(axis=0).A1

    vocabulary = vectorizer.vocabulary_
    ngrams = [gram for gram, idx in sorted(vocabulary.items(), key=lambda x: x[1])]

    sorted_indices = ngram_counts.argsort()[::-1]
    top_ngram_indices = sorted_indices[:num_top_ngrams]
    top_ngrams = [ngrams[idx] for idx in top_ngram_indices]
    top_ngram_counts = ngram_counts[top_ngram_indices]

    plt.figure(figsize=(10, 6))
    plt.barh(range(num_top_ngrams), top_ngram_counts, align='center', color='skyblue')
    plt.yticks(range(num_top_ngrams), top_ngrams)
    plt.xlabel('Count')
    plt.ylabel('N-gram')
    plt.title(f'Top {num_top_ngrams} {n}-grams - N-gram Analysis for {category} Category')
    plt.tight_layout()
    plt.show()

# Пример использования для анализа топ 10 биграмм по категориям
categories = df['category'].unique()
for category in categories:
    print(f"N-gram Analysis for {category} Category:")
    category_texts = df[df['category'] == category]['headline']
    plot_ngrams(category_texts, n=2, num_top_ngrams=10, category=category)


    import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Предобработка текстов
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Удаление строк с пустыми заголовками или описаниями
df = df[(df['headline'] != "") & (df['short_description'] != "")]

# К описанию добавляем заголовок
df['short_description'] = df['headline'] + " " + df['short_description']

# Создание нового DataFrame для хранения выбранных записей
sampled_df = pd.DataFrame()

# Перебор всех уникальных категорий
categories = df['category'].unique()

for category in categories:
    category_df = df[df['category'] == category]

    # Если количество записей в категории меньше n, берем все записи, иначе берем n случайных записей
    if len(category_df) < 500:
        sampled_category_df = category_df
    else:
        sampled_category_df = category_df.sample(n=500, random_state=42)

    # Добавляем выбранные записи в новый DataFrame
    sampled_df = pd.concat([sampled_df, sampled_category_df], ignore_index=True)

# Оставляем только нужные столбцы
sampled_df = sampled_df[['short_description', 'category']]

# Применение предобработки к столбцу short_description
sampled_df['short_description'] = sampled_df['short_description'].apply(preprocess_text)

# Энкодирование категорий
label_encoder = LabelEncoder()
sampled_df['category_encoded'] = label_encoder.fit_transform(sampled_df['category'])

# Разделение данных на обучающую и тестовую выборки
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    sampled_df['short_description'].tolist(), sampled_df['category_encoded'].tolist(),
    test_size=0.2,  # 80-20 разделение
    random_state=42  # Для воспроизводимости
)

# Создание кастомного датасета
class NewsCategoryDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Создание датасетов и загрузчиков данных
train_dataset = NewsCategoryDataset(train_texts, train_labels)
eval_dataset = NewsCategoryDataset(eval_texts, eval_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=8, shuffle=False)

# Инициализация устройства
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Working on {}".format(device))

sampled_df.sample(5)

from torch.optim import AdamW
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
num_labels = len(sampled_df['category'].unique())
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

# Функция для оценки модели
def evaluate_model(eval_loader, model, device, label_encoder):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for texts, labels in eval_loader:
            inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = torch.tensor(labels).to(device)

            outputs = model(**inputs)
            _, predicted = torch.max(outputs.logits, 1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy on test set: {accuracy * 100:.2f}%")

    # Используем все классы из label_encoder, даже если они отсутствуют в тестовых данных
    cm = confusion_matrix(true_labels, predictions, labels=range(len(label_encoder.classes_)))
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.show()

    report = classification_report(
        true_labels,
        predictions,
        labels=range(len(label_encoder.classes_)),
        target_names=label_encoder.classes_,
        zero_division=0
    )
    print(report)

# Обучение модели
for epoch in range(7):  # Количество эпох обучения
    model.train()
    running_loss = 0.0
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{7}", leave=False)

    for texts, labels in train_loader_tqdm:
        # Токенизация текста
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Исправленное преобразование меток
        if isinstance(labels, torch.Tensor):
            labels = labels.detach().clone().to(device)
        else:
            labels = torch.tensor(labels, dtype=torch.long).to(device)  # Указание типа для стабильности

        optimizer.zero_grad()

        # Прямой проход
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss

        # Обратное распространение
        loss.backward()
        optimizer.step()

        # Обновление статистики
        running_loss += loss.item()
        train_loader_tqdm.set_postfix(loss=running_loss / (train_loader_tqdm.n + 1))  # Более точное среднее

    # Вывод среднего лосса по эпохе
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")  # Форматирование вывода

    # Оценка модели после каждой эпохи
    evaluate_model(eval_loader, model, device, label_encoder)

   # Сохранение обученной модели
model_path = "./saved_model"
model.save_pretrained(model_path)

# Сохранение токенизатора
tokenizer_path = "./saved_tokenizer"
tokenizer.save_pretrained(tokenizer_path)

# Для последующей загрузки
model_path = "/content/drive/MyDrive/bert_finetuned_model/model"
tokenizer_path = "/content/drive/MyDrive/bert_finetuned_model/tokenizer"

# Загрузка сохранённой модели
model = BertForSequenceClassification.from_pretrained(model_path)

# Загрузка сохранённого токенизатора
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# Оценка модели на тестовой выборке
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for texts, labels in eval_loader:
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = torch.tensor(labels).to(device)

        outputs = model(**inputs)
        _, predicted = torch.max(outputs.logits, 1)

        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


# Вычисление точности
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

# Матрица ошибок
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Отчёт о классификации
report = classification_report(true_labels, predictions, target_names=label_encoder.classes_)
print(report)


# Создание нового DataFrame для хранения выбранных записей
user_df = pd.DataFrame()

# Перебор всех уникальных категорий
categories = df['category'].unique()

for category in categories:
    category_df = df[df['category'] == category]

    # Если количество записей в категории меньше n, берем все записи, иначе берем n случайных записей
    if len(category_df) < 250:
        sampled_category_df = category_df
    else:
        sampled_category_df = category_df.sample(n=250)

    # Добавляем выбранные записи в новый DataFrame
    user_df = pd.concat([user_df, sampled_category_df], ignore_index=True)

# user_df = df[df['category'] == "POLITICS"].sample(n=100, random_state=42)

# Оставляем только нужные столбцы
user_df = user_df[['short_description', 'category']]

# Применение предобработки к столбцу short_description
user_df['short_description'] = user_df['short_description'].apply(preprocess_text)

# Энкодирование категорий
label_encoder = LabelEncoder()
user_df['category_encoded'] = label_encoder.fit_transform(user_df['category'])
user_df.sample(5)

# Создание кастомного датасета
user_texts = user_df['short_description'].tolist()
user_labels = user_df['category_encoded'].tolist()

user_dataset = NewsCategoryDataset(user_texts, user_labels)
user_loader = DataLoader(user_dataset, batch_size=8, shuffle=False)


# Оценка модели на пользовательской выборке
model.eval()
user_predictions = []
user_true_labels = []

with torch.no_grad():
    for texts, labels in user_loader:
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = torch.tensor(labels).to(device)

        outputs = model(**inputs)
        _, predicted = torch.max(outputs.logits, 1)

        user_predictions.extend(predicted.cpu().numpy())
        user_true_labels.extend(labels.cpu().numpy())

# Вычисление точности
user_accuracy = accuracy_score(user_true_labels, user_predictions)
print(f"Accuracy on user set: {user_accuracy * 100:.2f}%")

# Матрица ошибок
user_cm = confusion_matrix(user_true_labels, user_predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(user_cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Отчёт о классификации
user_report = classification_report(user_true_labels, user_predictions, target_names=label_encoder.classes_)
print(user_report)


# Функция для предсказания категории для нового текста
def predict_category(text):
    text = preprocess_text(text)
    print(f"Preprocessed text: {text}")
    model.eval()
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        _, predicted = torch.max(outputs.logits, 1)

    predicted_category = label_encoder.inverse_transform(predicted.cpu().numpy())
    return predicted_category[0]

# Пример использования функции предсказания
example_text = "yummy sandwiches and cola are perfect for a lunch in sunny new york"
example = df.sample()
# Извлекаем значение столбца "short_description" для этого примера
# example_text = example['short_description'].values[0]
print("Example text:", example_text)
predicted_category = predict_category(example_text)
# real_category = example['category'].values[0]
# print(f"Real category: {real_category}")
print(f"Predicted category: {predicted_category}")









Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 3, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 33, in <module>
    import requests
  File "/usr/local/lib/python3.11/dist-packages/requests/__init__.py", line 43, in <module>
    import urllib3
  File "/usr/local/lib/python3.11/dist-packages/urllib3/__init__.py", line 15, in <module>
    from ._base_connection import _TYPE_BODY
  File "/usr/local/lib/python3.11/dist-packages/urllib3/_base_connection.py", line 5, in <module>
    from .util.connection import _TYPE_SOCKET_OPTIONS
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/__init__.py", line 5, in <module>
    from .request import SKIP_HEADER, SKIPPABLE_HEADERS, make_headers
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/request.py", line

FileNotFoundError: File /content/News_Category_Dataset_v3.json does not exist

# Новый раздел